# Introduction

This tutorial demonstrates angle resolved virtual backscatter electron
(BSE) imaging via signal extraction from raw electron backscatter diffraction
(EBSD) patterns.

For learning purposes, we recommend to use this notebook alongside our [user
guide](https://kikuchipy.org).

The data consists of 29 800 EBSD patterns from recrystallized polycrystalline
Nickel. It is available [via Zenodo](https://zenodo.org/record/3265037).

This functionality has been checked to run in kikuchipy-0.2.0 (May 2020). Bugs
are always possible, do not trust the code blindly, and if you experience any
issues, report them [in our issue
tracker](https://github.com/kikuchipy/kikuchipy-demos/issues).

Access all methods' documentation (docstrings) when the cursor is within the
function parantheses, like `function(CURSOR HERE)`, by holding `SHIFT` and
pressing `TAB`.

# Contents

1. [Loading and data inspection](#1)
2. [Background correction](#2)
3. [Interactive plotting](#3)  
    1. [Imaging](#3.1)  
    2. [Select a data region](#3.2)
4. [Multiple images from detector grid tiles](#4)
5. [RGB images](#5)  
    1. [From detector grid tiles](#5.1)  
    2. [From Gaussian fitting parameters](#5.2)

Set the interactive [Matplotlib backend](https://matplotlib.org/3.2.1/tutorials/introductory/usage.html#backends)
(`%matplotlib qt5`), import kikuchipy and other necessary libraries

In [ ]:
%matplotlib qt5
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp

# 1. Loading and data inspection <a class="anchor" id="1"></a>

Load the EBSD data

In [ ]:
data = "/home/hakon/phd/data/ni/2020/1/nordif/Pattern.dat"
s = kp.load(data, lazy=False)  # lazy=True to not load into memory before calling s.compute()

Create a copy of the raw patterns for part 5.B.

In [ ]:
s0 = s.deepcopy()

Inspect the metadata

In [ ]:
s.metadata

Inspect the signal (detector) and navigation (sample) axes

In [ ]:
s.axes_manager

Plot the patterns

In [ ]:
s.plot()

# 2. Background correction <a class="anchor" id="2"></a>

Remove the static background while keeping retain the relative intensities, thus
retaining the variation in the the backscatter electron yield from sample point
to sample point

In [ ]:
s.remove_static_background(relative=True)

# 3. Interactive plotting <a class="anchor" id="3"></a>

## 3.1 Imaging <a class="anchor" id="3.1"></a>

Plot an interactive virtual image by integrating the intensities within a rectangular subset of pixels on the detector

In [ ]:
roi = hs.roi.RectangularROI(left=0, top=0, right=10, bottom=10)

In [ ]:
s.plot_virtual_bse_intensity(roi)

Get the virtual BSE image associated with the last integration window used interactively

In [ ]:
vbse = s.get_virtual_bse_intensity(roi)

In [ ]:
vbse

Plot the virtual BSE image

In [ ]:
vbse.plot()

Inspect the metadata

In [ ]:
vbse.metadata

Save the virtual BSE image as a 32-bit TIFF file

In [ ]:
vbse.change_dtype(np.float32)

In [ ]:
vbse.save("vbse1.tif")

## 3.2 Select a data region <a class="anchor" id="3.2"></a>

Plot the data with an adjustable marker indicating where to crop the data region

In [ ]:
reg = hs.roi.RectangularROI(left=0, top=0, right=10, bottom=10)
s.plot()
reg.add_widget(s)

Crop the data set based on the region defined above

In [ ]:
s2 = reg(s)

Calculate the mean diffraction pattern from the selected region

In [ ]:
s2_mean = s2.mean(axis=(0, 1))

Plot the mean diffraction pattern from the selected region

In [ ]:
s2_mean.plot()

# 4. Multiple images from detector grid tiles <a class="anchor" id="4"></a>

Initialize a VirtualBSEImageGenerator

In [ ]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)

In [ ]:
vbse_gen

Inspect the generator detector grid

In [ ]:
vbse_gen.grid_shape

In [ ]:
vbse_gen.plot_grid(visible_indices=True, color="r")  # Default

Change the grid shape

In [ ]:
vbse_gen.grid_shape = (7, 7)

Compute one virtual BSE image per detector grid tile

In [ ]:
vbse_grid_images = vbse_gen.get_images_from_grid()

In [ ]:
vbse_grid_images

Plot the virtual BSE images

In [ ]:
vbse_grid_images.plot()

Rescale image intensities to between [-1, 1] before saving

In [ ]:
vbse_grid_images.rescale_intensity()

# Contrast stretching
#vbse_grid_images.rescale_intensity(percentiles=(0.5, 99.5))  

Save the BSE images as a 32-bit TIFF stack

In [ ]:
vbse_grid_images.unfold_navigation_space()

In [ ]:
vbse_grid_images

In [ ]:
vbse_grid_images.save("vbse2.tif")

# 5. RGB images <a class="anchor" id="5"></a>

## 5.1 From detector grid tiles <a class="anchor" id="5.1"></a>

Create an RGB image by coloring images produced from intensities within three
grid tiles by coloring them red, green and blue

In [ ]:
red = (6, 1)
green = (6, 3)
blue = (6, 5)
vbse_rgb_image = vbse_gen.get_rgb_image(r=red, g=green, b=blue)

In [ ]:
vbse_rgb_image

Plot the RGB image

In [ ]:
vbse_rgb_image.plot()

Plot the grid again, coloring the tiles used to produce the RGB image

In [ ]:
vbse_gen.plot_grid(rgb_channels=[red, green, blue], visible_indices=False)

Use the RGB image to navigate the patterns with

In [ ]:
s.plot(navigator=vbse_rgb_image)

Save the virtual BSE RGB image to an image file

In [ ]:
vbse_rgb_image.save("rgb.png")

To add an "alpha channel" (multiply RGB channels with a grey scale image) to the
RGB image, we compute the image quality

In [ ]:
s2 = s.deepcopy()

In [ ]:
s2.remove_dynamic_background()

In [ ]:
iq = s2.get_image_quality()

The alpha channel is scaled to the full black-white range. The darkest parts
might be a bit too dark, so we stretch the contrast

In [ ]:
vbse_rgba_image = vbse_gen.get_rgb_image(
    r=red,
    g=green,
    b=blue,
    alpha=iq,
    percentiles=(2, 98)  # Contrast stretching
)

Plot the RGBA image

In [ ]:
vbse_rgba_image.plot()

Save the virtual BSE RGBA image to an image file

In [ ]:
vbse_rgba_image.save("rgba.png")

## 5.2 From Gaussian fitting parameters <a class="anchor" id="5.2"></a>

See Britton, T Ben, Goran, Daniel, Tong, Vivian S, "Space rocks and optimising scanning electron channelling contrast," *Materials Characterization* **142** (2018), doi: https://doi.org/10.1016/j.matchar.2018.06.001 for further explanations of this part. 

Fit a 2D Gaussian to every raw pattern, by first creating a HyperSpy model

In [ ]:
m = s0.create_model()

In [ ]:
m

Add a 2D Gaussian component to the model

In [ ]:
m.append(hs.model.components2D.Gaussian2D())

In [ ]:
m.components

Inspect the model parameters

In [ ]:
m[0].parameters

Fit the model to the data (find the parameters describing the background signal
best for each pattern). Note that this might take a while...

In [ ]:
m.multifit()

Get maps of the fitted parameters, using the notation from the above mentioned
paper

In [ ]:
gaussian = m[0]

xc = gaussian.centre_x.as_signal().data
yc = gaussian.centre_x.as_signal().data
xw = gaussian.sigma_x.as_signal().data
yw = gaussian.sigma_y.as_signal().data
I = gaussian.A.as_signal().data

Perform contrast stretching on the results to remove outliers

In [ ]:
percentiles = (1, 99)

In [ ]:
p_xc = np.percentile(xc, q=percentiles)
xc2 = kp.pattern.rescale_intensity(xc, in_range=p_xc)

In [ ]:
p_yc = np.percentile(yc, q=percentiles)
yc2 = kp.pattern.rescale_intensity(yc, in_range=p_yc)

In [ ]:
p_xw = np.percentile(xw, q=percentiles)
xw2 = kp.pattern.rescale_intensity(xw, in_range=p_xw)

In [ ]:
p_yw = np.percentile(yw, q=percentiles)
yw2 = kp.pattern.rescale_intensity(yw, in_range=p_yw)

In [ ]:
p_I = np.percentile(I, q=percentiles)
I2 = kp.pattern.rescale_intensity(I, in_range=p_I)

Create RGB images from the parameters

In [ ]:
rgb_xc_yc_I_array = kp.generators.util.get_rgb_image(channels=[xc2, yc2, I2])

In [ ]:
rgb_xw_yw_I_array = kp.generators.util.get_rgb_image(channels=[xw2, yw2, I2])

Plot the results

In [ ]:
fig, ax = plt.subplots(ncols=2)
ax[0].imshow(rgb_xc_yc_I_array)
ax[1].imshow(rgb_xw_yw_I_array)

Save the results to file

In [ ]:
plt.imsave("rgb_xc_yc_I.png", rgb_xc_yc_I_array)
plt.imsave("rgb_xw_yw_I.png", rgb_xw_yw_I_array)